!pip install torch torchaudio sounddevice transformers
pip install pyaudio
brew install portaudio


In [1]:
import sounddevice as sd
import numpy as np
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
#from GPU_torch import GPU

/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#device = GPU()
#print("current device : ",device)
#print(device)

In [3]:

import torch
import subprocess

def GPU():
    if torch.cuda.is_available() == True:
        device = 'cuda'
        templist = [1, 2, 3]
        templist = torch.FloatTensor(templist).to(device)
        print("Cuda torch working : ", end="")
        print(templist.is_cuda)
        print("current device no. : ", end="")
        print(torch.cuda.current_device())
        print("GPU device count : ", end="")
        print(torch.cuda.device_count())
        print("GPU name : ", end="")
        print(torch.cuda.get_device_name(0))
        print("device : ", device)
        # Execute the nvidia-smi command using subprocess
        try:
            output = subprocess.check_output(['nvidia-smi']).decode('utf-8')
            print("nvidia-smi output:")
            print(output)
        except (subprocess.CalledProcessError, FileNotFoundError) as e:
            print("Error executing nvidia-smi command:", str(e))
    elif torch.backends.mps.is_available() == True:
        print("Apple device detected\nActivating Apple Silicon GPU")
        device = torch.device("mps")
    else:
        print("cant use gpu , activating cpu")
        device = 'cpu'

    return device
device = GPU()
device = 'cpu'
print(device)

Apple device detected
Activating Apple Silicon GPU
cpu


In [4]:
import pyaudio
import numpy as np
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


In [5]:
# Pretrained model loading
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

# Check if CUDA is available and set the device accordingly
model.to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=1024, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder

In [6]:
# pyaudio params
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000

# speech recognition function
def recognize_speech_from_mic():
    p = pyaudio.PyAudio()
    frames = []
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")
    for i in range(0, int(RATE / CHUNK * 5)):
        data = stream.read(CHUNK)
        frames.append(np.frombuffer(data, dtype=np.int16))

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()


    frame = np.concatenate(frames).astype(np.float32)
    input_values = tokenizer(frame, return_tensors="pt", padding="longest", sampling_rate=RATE).input_values
    input_values = input_values.to(device)



    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.decode(predicted_ids[0])
    print("Transcription: ", transcription)


In [7]:
if __name__ == "__main__":
    recognize_speech_from_mic()


* recording
* done recording
Transcription:  HALLO THIS IS RECORDING FRO


## BART (translation not finished)

In [8]:
# import sounddevice as sd
# import numpy as np
# import torch
# import torchaudio
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, BertModel, BertTokenizer

# # Pretrained model loading
# tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

# # Korean BERT Model
# kor_bert_tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
# kor_bert_model = BertModel.from_pretrained("beomi/kcbert-base")

# # Check if CUDA is available and set the device accordingly
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# kor_bert_model.to(device)

# # pyaudio params
# CHUNK = 1024
# FORMAT = pyaudio.paInt16
# CHANNELS = 1
# RATE = 16000

# # speech recognition function
# def recognize_speech_from_mic():
#     p = pyaudio.PyAudio()
#     frames = []
#     stream = p.open(format=FORMAT,
#                     channels=CHANNELS,
#                     rate=RATE,
#                     input=True,
#                     frames_per_buffer=CHUNK)

#     print("* recording")
#     for i in range(0, int(RATE / CHUNK * 5)):
#         data = stream.read(CHUNK)
#         frames.append(np.frombuffer(data, dtype=np.int16))

#     print("* done recording")

#     stream.stop_stream()
#     stream.close()
#     p.terminate()

#     frame = np.concatenate(frames).astype(np.float32)
#     input_values = tokenizer(frame, return_tensors="pt", padding="longest", sampling_rate=RATE).input_values
#     input_values = input_values.to(device)

#     with torch.no_grad():
#         logits = model(input_values).logits

#     predicted_ids = torch.argmax(logits, dim=-1)
#     transcription = tokenizer.decode(predicted_ids[0])

#     # Tokenize the transcription and pass through Korean BERT model
#     inputs = kor_bert_tokenizer(transcription, return_tensors="pt")
#     outputs = kor_bert_model(**inputs)

#     print("Transcription: ", transcription)

# if __name__ == "__main__":
#     recognize_speech_from_mic()
